In [1]:
# Reading the csv file for gRNA for sars cov2
import pandas as pd 
import re
import numpy as np
import os
import subprocess
# importing biopython
#from Bio import SeqIO

In [2]:
import time
start = time.time()


In [3]:
df = pd.read_csv("Input/CovidCasRxguides.csv")
df.head()

,GuideName,GuideSeq,MatchPos,GuideScores,Rank,standardizedGuideScores,quartiles
0,crRNA00001:1-23,CTGGGAAGGTATAAACCTTTAAT,23,0.020537,0.1789,0.170280,2
1,crRNA00002:2-24,CCTGGGAAGGTATAAACCTTTAA,24,0.013715,0.1703,0.166492,2
2,crRNA00003:3-25,ACCTGGGAAGGTATAAACCTTTA,25,0.017811,0.1754,0.168766,2
3,crRNA00004:4-26,TACCTGGGAAGGTATAAACCTTT,26,0.028522,0.1889,0.174715,2
4,crRNA00005:5-27,TTACCTGGGAAGGTATAAACCTT,27,0.019235,0.1772,0.169557,2


In [5]:
# Checking the number of rows
len(df)

28749

In [6]:
## Finding the top quartile 
ng = len(df)# number of guide candidates

df = df.sort_values(by=["standardizedGuideScores"], ascending=False)

In [7]:
n = ng//4 # To get the top quartile
df = df[:n] 

In [8]:
# After sorting the length of dataframe
len(df)

7187

In [9]:
gRNA_list = list(df["GuideSeq"])
GuideName_list = list(df["MatchPos"])
score_list = list(df["standardizedGuideScores"])
# Function to compare two strings
def Str2MatchStr1(str1, str2):
    count = 0
    l = len(str1)
    for i in range(l):
        if str1[i]== str2[i]:
            count = count+1
    return count

match1 = []
match2 = []
similar_nt = []
score_gRNA_A = []
score_gRNA_B = []
GuideName_A = []
GuideName_B = []
l = len(gRNA_list)

for i in range(l-1):
    for j in range(i+1,l):
        #print(i,j)
            x = gRNA_list[i]
            row = gRNA_list[j]
            score1 = score_list[i]
            score2 = score_list[j]
            guideA = GuideName_list[i]
            guideB = GuideName_list[j]
            matchnumber = Str2MatchStr1(x, row) 
            if (matchnumber >= 17):
                match1.append(x)
                match2.append(row)
                similar_nt.append(matchnumber)
                score_gRNA_A.append(score1)
                score_gRNA_B.append(score2)
                GuideName_A.append(guideA)
                GuideName_B.append(guideB)
                #print('position', i,'gRNA', x, ' and ','position',j, 'match', row, 'similar nt', matchnumber)

In [10]:
df_pairs = pd.DataFrame({'GuideA Position': GuideName_A,'Guide A':match1,'Guide A Score': score_gRNA_A,'GuideB Position': GuideName_B,'Guide B': match2, 'Guide B Score': score_gRNA_B,'Similar NT': similar_nt})
df_pairs.head()

,GuideA Position,Guide A,Guide A Score,GuideB Position,Guide B,Guide B Score,Similar NT
0,9823,ATTTAACAAAAAGGTGCACAGCG,1.000000,10539,ATGTAACAAAAAGAGACACAGTC,0.896787,17
1,15133,TACTACAGATAGAGACACCAGCT,1.000000,9266,TAGTAGATACACAAACACCAGCT,0.932930,17
2,18736,ATCCAATAGAATGATGCCAACAG,1.000000,19510,ACTCATTAGCATGATGTCTACAG,0.644632,17
3,9825,TTATTTAACAAAAAGGTGCACAG,0.995615,10541,GCATGTAACAAAAAGAGACACAG,0.988901,17
4,10541,GCATGTAACAAAAAGAGACACAG,0.988901,11193,GAAGGTAACAAAAACAAACAGAG,0.558760,18


In [11]:
len(df_pairs)

205

In [12]:
#Reverse complements a given string
def revcom(s):
    basecomp = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A','U':'A'}
    letters = list(s[::-1])
    letters = [basecomp[base] for base in letters]
    return ''.join(letters)

In [13]:
## Doing Sequence Complement of Homologous Guide Sequences to find the Target position in SARS Cov2 genome
target_A_list = df_pairs["Guide A"].apply(revcom)
target_B_list = df_pairs["Guide B"].apply(revcom)

df_pairs["Target A"] = target_A_list
df_pairs["Target B"] = target_B_list

In [14]:
df_pairs.head()

,GuideA Position,Guide A,Guide A Score,GuideB Position,Guide B,Guide B Score,Similar NT,Target A,Target B
0,9823,ATTTAACAAAAAGGTGCACAGCG,1.000000,10539,ATGTAACAAAAAGAGACACAGTC,0.896787,17,CGCTGTGCACCTTTTTGTTAAAT,GACTGTGTCTCTTTTTGTTACAT
1,15133,TACTACAGATAGAGACACCAGCT,1.000000,9266,TAGTAGATACACAAACACCAGCT,0.932930,17,AGCTGGTGTCTCTATCTGTAGTA,AGCTGGTGTTTGTGTATCTACTA
2,18736,ATCCAATAGAATGATGCCAACAG,1.000000,19510,ACTCATTAGCATGATGTCTACAG,0.644632,17,CTGTTGGCATCATTCTATTGGAT,CTGTAGACATCATGCTAATGAGT
3,9825,TTATTTAACAAAAAGGTGCACAG,0.995615,10541,GCATGTAACAAAAAGAGACACAG,0.988901,17,CTGTGCACCTTTTTGTTAAATAA,CTGTGTCTCTTTTTGTTACATGC
4,10541,GCATGTAACAAAAAGAGACACAG,0.988901,11193,GAAGGTAACAAAAACAAACAGAG,0.558760,18,CTGTGTCTCTTTTTGTTACATGC,CTCTGTTTGTTTTTGTTACCTTC


In [15]:
def Mismatch_Alignment(a,b):
    mismatch = dif(a,b)
    for val in mismatch:
        list1 = list(a)
        list1[val] = 'X'
        a = ''.join(list1)

    return(a)

In [16]:
def dif(a, b):
    return [i for i in range(len(a)) if a[i] != b[i]]

In [17]:
Mismatch_position_list = []
for i in range(len(df_pairs)):
    a = df_pairs["Target A"][i]
    b = df_pairs["Target B"][i]
    Mismatch_position = Mismatch_Alignment(a,b)
    Mismatch_position_list.append(Mismatch_position)

In [18]:
df_pairs["Mismatch_Position"] = Mismatch_position_list

In [19]:
df_pairs.to_csv (r'df_pairs_sorted_top_quartile.csv', index = False, header=True)

In [20]:
# Creating Decimal tobase 4 function in python
def is_zero(n):
    for d in n:
        if d != 0:
            return False
    return True

def modulo_div(n,original_base,destination_base):
    carry = 0
    for i in range(len(n)):
        d = n[i]
        d+=original_base*carry 
        carry = d%destination_base 
        d=(d//destination_base)
        n[i] = d
    return carry

def convertBase(n,original_base,destination_base):
    digits = []    
    while not is_zero(n):
        digits.insert(0,modulo_div(n,original_base,destination_base))
    return digits

def Dec2base4(inputNum):
    if inputNum <4:
        l1 = []
        num = inputNum//1
        l1.append(num)
    elif inputNum >= 4:
        n = str(inputNum)
        b = str(n)
        c = []

        for digit in b:
            c.append (int(digit))
        n = c
        original_base = 10
        destination_base = 4
        l1 = convertBase(n,original_base,destination_base)             
    return l1

### Design of optimised PgRNA

#### GC Content

In [21]:
def dif(a, b):
    return [i for i in range(len(a)) if a[i] != b[i]]

# Checking all the criteria for gRNA design
# GC content > 30% and <70%

# Defining a function
# GC content (or Guanine-Cytosine content)
def gc_content(genome):
    g_content = genome.count("G")
    c_content = genome.count("C")
    a_content = genome.count("A")
    t_content = genome.count("T")
    return 100*((g_content+c_content)/(a_content+c_content+g_content+t_content))

#### CFD Scoring part

In [4]:
df_mismatch = pd.read_csv("Input/Mismatch_Scores.csv")
#df_mismatch

In [5]:
df_mismatch_scores = df_mismatch.set_index('Pair')
df_mismatch_scores

,1,2,3,4,5,6,7,8,9,10,...,18,19,20,21,22,23,24,25,26,27
Pair,,,,,,,,,,,,,,,,,,,,,
tG:sU,0.974520,0.693669,0.815164,0.867713,0.858668,0.669409,0.812718,0.600761,0.656990,0.772818,...,0.902661,1.190228,1.038503,0.780292,0.968315,0.788823,0.833326,0.717546,1.073300,0.870825
tC:sU,0.952303,1.034247,0.863837,1.133359,0.559244,0.872666,0.859019,0.660549,0.674264,0.679772,...,0.774293,0.908869,0.782694,0.879373,0.938191,0.753834,0.825366,0.824055,0.927345,0.937318
tA:sU,0.877519,1.018451,1.048960,0.862794,0.771068,0.779113,0.920649,0.855728,0.888095,0.502231,...,0.832402,0.894867,1.045077,1.437197,1.054614,0.852128,0.831014,0.857205,0.822636,1.143781
tU:sG,1.065876,1.076388,0.892729,0.868821,1.088882,0.908753,0.669460,0.690120,0.705541,0.636520,...,0.902864,0.941087,0.729613,0.762545,0.820029,0.833169,0.944286,0.929217,0.862516,1.047475
tC:sG,0.977541,0.906344,0.917680,0.871254,0.875400,0.952794,0.725404,0.737952,0.770026,0.685226,...,0.996722,0.702591,1.006903,0.978842,0.735509,0.788219,0.919202,0.828628,0.897072,0.962061
tA:sG,0.997148,0.958448,1.032647,1.008273,1.120190,0.922862,0.840220,0.808145,0.725025,0.702026,...,0.953046,0.781027,0.970779,0.996075,0.864480,1.031756,1.237275,1.128326,0.842466,0.856491
tU:sC,0.962548,1.110817,0.944027,1.092226,0.914208,0.900019,0.849311,1.106109,0.846656,0.654116,...,0.976502,1.097581,1.006110,0.835043,0.874378,0.987515,1.137065,0.963879,0.846670,1.150866
tG:sC,1.109798,1.008719,1.082349,0.922670,0.849772,0.967784,0.879848,0.860344,0.832122,0.714449,...,0.800116,1.053971,0.953604,0.997356,0.958687,1.197421,1.084419,0.812982,1.138933,1.139552
tA:sC,0.936904,0.897960,1.037448,1.010420,0.941216,0.978227,1.441710,0.898812,0.977896,0.720228,...,1.048519,1.031907,1.071964,1.087671,0.947610,0.889467,0.943664,1.137739,1.073230,1.334418


In [6]:
df_adjacent_mismatch = pd.read_csv("Input/Adjacent_Mismatch.csv")
df_adjacent_mismatch

,Position,Double,Triple
0,27,0.932294,0.920280
1,26,0.910267,0.804107
2,25,0.804133,0.767498
3,24,0.766854,0.764801
4,23,0.852079,0.720960
5,22,0.809224,0.740623
6,21,0.837299,0.748435
7,20,0.880066,0.748435
8,19,0.844483,0.738082
9,18,0.825985,0.733910


In [25]:
#Reverse complements a given string
def revcom(s):
    basecomp = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A','U':'A'}
    letters = list(s[::-1])
    letters = [basecomp[base] for base in letters]
    return ''.join(letters)

In [26]:
## Finding the mismatch and calculating scores for single mismatch
def calc_cfd(wt,sg):
    M_1 = Find_single_Mismatch(wt,sg)
    #print("The position of Single Mismatch is",M_1)
    score = 1
    sg = revcom(sg)
    wt = revcom(wt)
    sg = sg.replace('T','U')
    wt = wt.replace('T','U')
    #print("Reverse Complement sg:", sg)
    #print("sg:", sg)
    #print("DNA", wt)
    s_list = list(sg)
    wt_list = list(wt)
    #print(s_list)
    #print(wt_list)
    for i in M_1:
        j = len(sg)-(i+1)
        key = 't'+wt_list[j]+':s'+s_list[j]
        #print(key)
        position = str(i+1)
        #print(position)
        #print(df_mismatch_scores[position][key])
        score*= df_mismatch_scores[position][key]
    #print("Mismatch Penalty Score due to single Mismatch is:", score)
    return score

In [27]:
def Find_Triple_Mismatch(a,b):
    a = a
    b = b
    ### Combining all
    def dif(a, b):
        return [i for i in range(len(a)) if a[i] != b[i]]
    mismatch = dif(a,b)
    for val in mismatch:
        list1 = list(a)
        list1[val] = 'X'
        a = ''.join(list1)
    import re
    l_3 = [m.start() for m in re.finditer('XXX', a)]
    return l_3

In [28]:
def Find_Double_Mismatch(a,b):
    a = a
    b = b
    ### Combining all
    def dif(a, b):
        return [i for i in range(len(a)) if a[i] != b[i]]
    mismatch = dif(a,b)
    for val in mismatch:
        list1 = list(a)
        list1[val] = 'X'
        a = ''.join(list1)
    import re
    l_3 = [m.start() for m in re.finditer('XXX', a)]
    l_2 = [m.start() for m in re.finditer('XX', a)]
    def Diff(li1, li2): 
        li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2] 
        return li_dif
    l = Diff(l_2,l_3)
    return l

In [29]:
def Find_single_Mismatch(a,b):
    a = a
    b = b
    ### Combining all
    def dif(a, b):
        return [i for i in range(len(a)) if a[i] != b[i]]
    mismatch = dif(a,b)
    for val in mismatch:
        list1 = list(a)
        list1[val] = 'X'
        a = ''.join(list1)
    import re
    l1_3 = [m.start() for m in re.finditer('XXX', a)]
    s1 = []
    for val in l1_3:
        #print(val)
        x = val+1
        y = val+2
        s = [x,y]
        s1.append(s)
    l1_3 = l1_3 + s1

    # function used for removing nested  
    # lists in python.
    output = [] 
    def reemovNestings(l):  
        for i in l: 
            if type(i) == list: 
                reemovNestings(i) 
            else: 
                output.append(i) 
        return output

    l1_3 = reemovNestings(l1_3) 

    l1_2 = [m.start() for m in re.finditer('XX', a)]
    s1 = []
    for val in l1_2:
        x = val+1
        s1.append(x)
    l1_2 = l1_2 + s1

    l1_consecutive = l1_3 + l1_2
    l1_consecutive = list(set(l1_consecutive))

    l1_1 = [m.start() for m in re.finditer('X', a)]


    def Diff(li1, li2): 
        li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2] 
        return li_dif


    l = Diff(l1_1,l1_consecutive)
    
    return l


In [30]:
def Cal_Mismatch_Score(a,b):
    M_3 = Find_Triple_Mismatch(a,b)
    M_2 = Find_Double_Mismatch(a,b)
    M_1 = Find_single_Mismatch(a,b)

    score2 = 1
    score3 = 1
    score_1 = 1
    if (len(M_3) != 0) :
        for i in M_3:
            #print("The position of Triple mismatch is",i)
            j = 27-(i+1)
            score3 = score3*(df_adjacent_mismatch["Triple"][j])*(df_adjacent_mismatch["Triple"][j-1])*(df_adjacent_mismatch["Triple"][j-2])
    #print("The mismatch penalty due to Triple mismatch is",score3)
    if (len(M_2) != 0) :
        for i in M_2:
            #print("The position of Double mismatch is",i)
            k = 27-(i+1)
            score2 = score2*(df_adjacent_mismatch["Double"][k])*(df_adjacent_mismatch["Double"][k-1])
    #print("The mismatch penalty due to double mismatch is",score2)
    if (len(M_1) != 0) :
        #print(len(M_1))
        score_1 = calc_cfd(a,b)
        #print(score_1)
        #print("The mismatch penalty due to single mismatch is",score_1)
    #print(type(score_1))
    #print('score_1', score_1)
    Net_Score = float(score2)*float(score3)*float(score_1)
    return Net_Score

### Design of optimised PgRNA based on target activity at A and B greater than minimum of both columns Spacer A and B 

In [31]:
## Design of optimised PgRNA Based on target activity at target A and B both > 0.75
l = len(df_pairs)
crRNA_list = []
Guide_score = []
Guide_scoreA = []
Guide_scoreB = []
for i in range(l):
    s1 = df_pairs["Guide A"][i]
    t1 = s1# storing as t1 because we are changing the s1 for the design of pgRNAs
    s2 = df_pairs["Guide B"][i]
    t2 = s2# storing as t2 because we are changing the s2 for the design of pgRNAs
    s1_score = df_pairs["Guide A Score"][i]
    s2_score = df_pairs["Guide B Score"][i]

    mismatch = dif(s1, s2)
    MMn = len(mismatch)
    #print(MMn)
    Com_MMn = ((4**MMn))
    BaseSwap_list = []
    for n in range(Com_MMn):
        #print(n)
        BaseSwap = Dec2base4(n);
        #print(BaseSwap);
        #print(type(BaseSwap))
        BaseSwap_list.append(BaseSwap)
        
    for val in BaseSwap_list:
        if(len(val) < MMn):
            zero_padding = MMn - len(val)
            listofzeros = [0]*zero_padding
            for n in listofzeros:
                val.insert(0, n)
            #print(val)
    #print("For Mismatch", MMn,"The list of replacer", BaseSwap_list)
    
    
    for pair in BaseSwap_list:
        l = len(pair)
        for j in range(l):
            if (pair[j] == 0):
                pair[j] = "A"
            elif (pair[j] == 1):
                pair[j] = "C"
            elif (pair[j] == 2):
                pair[j] = "G"
            elif (pair[j] == 3):
                pair[j] = "T"
        #print(pair)
        
    crRNA_list_1 = []
    for replacer in BaseSwap_list:
        i = 0
        for val in mismatch:
            #print(replacer[i])
            list1 = list(s1)
            list1[val] = replacer[i]
            i = i+1
            s1 = ''.join(list1)

        #print(s1)
        crRNA_list_1.append(s1)
        
        
    ## Checking the GC content of crRNA in CrRNA_list
    
    crRNA_list_1_corrected1 = []
    for i in range(len(crRNA_list_1)):
        gc_value = gc_content(crRNA_list_1[i])
        if (gc_value >30) and (gc_value < 70 ):
            crRNA_list_1_corrected1.append(crRNA_list_1[i])
            
            
    crRNA_list_1_corrected2 = []        
    # Criteria 2
    # To avoid repeats (≥4 consecutive ‘rU’ or ≥5 consecutive ‘rG’, ‘rC’, or ‘rA)
    l = len(crRNA_list_1_corrected1)
    for i in range(l):
        U_repeat = crRNA_list_1_corrected1[i].find('UUUU')
        G_repeat = crRNA_list_1_corrected1[i].find('GGGGG')
        C_repeat = crRNA_list_1_corrected1[i].find('CCCCC')
        A_repeat = crRNA_list_1_corrected1[i].find('AAAAA')
        if (U_repeat==-1) and (G_repeat==-1) and (C_repeat==-1) and (A_repeat==-1):
            crRNA_list_1_corrected2.append(crRNA_list_1_corrected1[i]) 
            
    #Checking the cfd cores
    Activity1 = min(df_pairs['Guide A Score'])  
    Activity2 = min(df_pairs['Guide B Score']) 
    crRNA_list_1_corrected3 = []
    Activity_guide = []
    Activity_guideA = []
    Activity_guideB = []
    for i in range(len(crRNA_list_1_corrected2)):
        mismatch_scoreA = Cal_Mismatch_Score(t1, crRNA_list_1_corrected2[i])
        Activity_scoreA  = s1_score* mismatch_scoreA
        mismatch_scoreB = Cal_Mismatch_Score(t2, crRNA_list_1_corrected2[i])
        Activity_scoreB  = s2_score* mismatch_scoreB
        
        Activity_guide_avg = (Activity_scoreA+Activity_scoreB)/2 # Taking the average of activity of Guide at target A and B
        
        if ( Activity_scoreA > Activity1) and (Activity_scoreB > Activity2):
            #print(mismatch_scoreA)
            #print(mismatch_scoreA)
            crRNA_list_1_corrected3.append(crRNA_list_1_corrected2[i])
            Activity_guide.append(Activity_guide_avg)
            Activity_guideA.append(Activity_scoreA)
            Activity_guideB.append(Activity_scoreB)
            
    ## Appending to final design RNA list
    d2 = {'Guide A' : t1, 'Guide B' : t2, 'Mismatch' : MMn , 'pgRNA List' : crRNA_list_1_corrected3}
    crRNA_list.append(d2)
    ## Appending the guide scores
    Guide_score.append(Activity_guide)
    Guide_scoreA.append(Activity_guideA)
    Guide_scoreB.append(Activity_guideB)
    
    

In [32]:
df_crRNA = pd.DataFrame(crRNA_list)
df_crRNA

,Guide A,Guide B,Mismatch,pgRNA List
0,ATTTAACAAAAAGGTGCACAGCG,ATGTAACAAAAAGAGACACAGTC,6,[]
1,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,6,"[TAATACAGACAAAGACACCAGCT, TAATACAGACAGAAACACCA..."
2,ATCCAATAGAATGATGCCAACAG,ACTCATTAGCATGATGTCTACAG,6,"[AAACAATAGCATGATGGCGACAG, AAACAATAGCATGATGGCTA..."
3,TTATTTAACAAAAAGGTGCACAG,GCATGTAACAAAAAGAGACACAG,6,[]
4,GCATGTAACAAAAAGAGACACAG,GAAGGTAACAAAAACAAACAGAG,5,[]
5,CAAATTGTGCAATTTGCGGCCAA,AAAATTGTACATTCAGCAGCCAA,6,"[AAAATTGTACAATATGCCGCCAA, AAAATTGTACAATATGCGGC..."
6,AAATTGTGCAATTTGCGGCCAAT,AAATTGTACATTCAGCAGCCAAA,6,"[AAATTGTACAATCTGCAGCCAAT, AAATTGTACAATCTGCGGCC..."
7,AAATTGTGCAATTTGCGGCCAAT,TAATTGTCCAACTTAGGGTCAAT,6,"[AAATTGTCCAACTTATGGACAAT, AAATTGTCCAATTTATGGGC..."
8,TAATAAATGTACTACCAGCACAG,AAGTAAATGTTGTACCATCACAC,6,"[AAATAAATGTAGTACCAACACAG, AAATAAATGTAGTACCACCA..."
9,TATTTAACAAAAAGGTGCACAGC,CATGTAACAAAAAGAGACACAGT,6,[]


In [33]:
x = 0
for i in range(len(df_crRNA)):
    #print(len(df_crRNA['pgRNA List'][i]))
    x = x + len(df_crRNA['pgRNA List'][i])
print(x)

29814


In [34]:
# Use list comprehension to convert the nested lists and get a full list of pgRNAs Activity Score
Guide_score_list = [ item for elem in Guide_score for item in elem]
len(Guide_score_list)

29814

In [35]:
# Use list comprehension to convert the nested lists and get a full list of pgRNAs Activity Score at Target A
Guide_scoreA_list = [ item for elem in Guide_scoreA for item in elem]
len(Guide_scoreA_list)

29814

In [36]:
# Use list comprehension to convert the nested lists and get a full list of pgRNAs Activity Score at Target A
Guide_scoreB_list = [ item for elem in Guide_scoreB for item in elem]
len(Guide_scoreB_list)

29814

In [37]:
df_crRNA = pd.DataFrame(crRNA_list)
df_crRNA

,Guide A,Guide B,Mismatch,pgRNA List
0,ATTTAACAAAAAGGTGCACAGCG,ATGTAACAAAAAGAGACACAGTC,6,[]
1,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,6,"[TAATACAGACAAAGACACCAGCT, TAATACAGACAGAAACACCA..."
2,ATCCAATAGAATGATGCCAACAG,ACTCATTAGCATGATGTCTACAG,6,"[AAACAATAGCATGATGGCGACAG, AAACAATAGCATGATGGCTA..."
3,TTATTTAACAAAAAGGTGCACAG,GCATGTAACAAAAAGAGACACAG,6,[]
4,GCATGTAACAAAAAGAGACACAG,GAAGGTAACAAAAACAAACAGAG,5,[]
5,CAAATTGTGCAATTTGCGGCCAA,AAAATTGTACATTCAGCAGCCAA,6,"[AAAATTGTACAATATGCCGCCAA, AAAATTGTACAATATGCGGC..."
6,AAATTGTGCAATTTGCGGCCAAT,AAATTGTACATTCAGCAGCCAAA,6,"[AAATTGTACAATCTGCAGCCAAT, AAATTGTACAATCTGCGGCC..."
7,AAATTGTGCAATTTGCGGCCAAT,TAATTGTCCAACTTAGGGTCAAT,6,"[AAATTGTCCAACTTATGGACAAT, AAATTGTCCAATTTATGGGC..."
8,TAATAAATGTACTACCAGCACAG,AAGTAAATGTTGTACCATCACAC,6,"[AAATAAATGTAGTACCAACACAG, AAATAAATGTAGTACCACCA..."
9,TATTTAACAAAAAGGTGCACAGC,CATGTAACAAAAAGAGACACAGT,6,[]


In [38]:
#df_pairs["Guide A"] == df_crRNA["Guide A"]

In [39]:
df_crRNA1 = df_crRNA
df_crRNA1['GuideA Position'] = df_pairs['GuideA Position']
df_crRNA1['Guide A Score'] = df_pairs['Guide A Score']
df_crRNA1['GuideB Position'] = df_pairs['GuideB Position']
df_crRNA1['Guide B Score'] = df_pairs['Guide B Score']
df_crRNA1 = df_crRNA1.rename(columns={"Guide A": "GuideA", "Guide B": "GuideB", "GuideA Position": "GuideAPosition", "GuideB Position": "GuideBPosition", "Guide A Score": "GuideAScore", "Guide B Score": "GuideBScore", "pgRNA List": "pgRNA"})
df_crRNA1

,GuideA,GuideB,Mismatch,pgRNA,GuideAPosition,GuideAScore,GuideBPosition,GuideBScore
0,ATTTAACAAAAAGGTGCACAGCG,ATGTAACAAAAAGAGACACAGTC,6,[],9823,1.000000,10539,0.896787
1,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,6,"[TAATACAGACAAAGACACCAGCT, TAATACAGACAGAAACACCA...",15133,1.000000,9266,0.932930
2,ATCCAATAGAATGATGCCAACAG,ACTCATTAGCATGATGTCTACAG,6,"[AAACAATAGCATGATGGCGACAG, AAACAATAGCATGATGGCTA...",18736,1.000000,19510,0.644632
3,TTATTTAACAAAAAGGTGCACAG,GCATGTAACAAAAAGAGACACAG,6,[],9825,0.995615,10541,0.988901
4,GCATGTAACAAAAAGAGACACAG,GAAGGTAACAAAAACAAACAGAG,5,[],10541,0.988901,11193,0.558760
5,CAAATTGTGCAATTTGCGGCCAA,AAAATTGTACATTCAGCAGCCAA,6,"[AAAATTGTACAATATGCCGCCAA, AAAATTGTACAATATGCGGC...",29195,0.986171,9030,0.963057
6,AAATTGTGCAATTTGCGGCCAAT,AAATTGTACATTCAGCAGCCAAA,6,"[AAATTGTACAATCTGCAGCCAAT, AAATTGTACAATCTGCGGCC...",29194,0.982935,9029,0.931580
7,AAATTGTGCAATTTGCGGCCAAT,TAATTGTCCAACTTAGGGTCAAT,6,"[AAATTGTCCAACTTATGGACAAT, AAATTGTCCAATTTATGGGC...",29194,0.982935,5961,0.609077
8,TAATAAATGTACTACCAGCACAG,AAGTAAATGTTGTACCATCACAC,6,"[AAATAAATGTAGTACCAACACAG, AAATAAATGTAGTACCACCA...",7661,0.981468,12536,0.874413
9,TATTTAACAAAAAGGTGCACAGC,CATGTAACAAAAAGAGACACAGT,6,[],9824,0.979314,10540,0.893182


In [40]:
df1 = df_crRNA1[['GuideA', 'pgRNA']].copy()
GuideAs = []
pgRNAs = []
for _, row in df1.iterrows():
    GuideA = row.GuideA
    
    for val in row.pgRNA:
        GuideAs.append(GuideA)
        pgRNAs.append(val)
df2 = pd.DataFrame({"GuideA": GuideAs,
    "pgRNA": pgRNAs
})
df2.head(10)

,GuideA,pgRNA
0,TACTACAGATAGAGACACCAGCT,TAATACAGACAAAGACACCAGCT
1,TACTACAGATAGAGACACCAGCT,TAATACAGACAGAAACACCAGCT
2,TACTACAGATAGAGACACCAGCT,TAATACAGACAGAGACACCAGCT
3,TACTACAGATAGAGACACCAGCT,TAATAGACACAAAGACACCAGCT
4,TACTACAGATAGAGACACCAGCT,TAATAGAGACAAAGACACCAGCT
5,TACTACAGATAGAGACACCAGCT,TAATAGAGACAGAGACACCAGCT
6,TACTACAGATAGAGACACCAGCT,TACTAAAGACAAAGACACCAGCT
7,TACTACAGATAGAGACACCAGCT,TACTACAAACAAAGACACCAGCT
8,TACTACAGATAGAGACACCAGCT,TACTACACACAAAGACACCAGCT
9,TACTACAGATAGAGACACCAGCT,TACTACAGACAAAGACACCAGCT


In [41]:
df1 = df_crRNA1[['GuideB', 'pgRNA']].copy()
GuideBs = []
pgRNAs = []
for _, row in df1.iterrows():
    GuideB = row.GuideB
    
    for val in row.pgRNA:
        GuideBs.append(GuideB)
        pgRNAs.append(val)
df2 = pd.DataFrame({"GuideB": GuideBs,
    "pgRNA": pgRNAs
})
df2.head(10)

,GuideB,pgRNA
0,TAGTAGATACACAAACACCAGCT,TAATACAGACAAAGACACCAGCT
1,TAGTAGATACACAAACACCAGCT,TAATACAGACAGAAACACCAGCT
2,TAGTAGATACACAAACACCAGCT,TAATACAGACAGAGACACCAGCT
3,TAGTAGATACACAAACACCAGCT,TAATAGACACAAAGACACCAGCT
4,TAGTAGATACACAAACACCAGCT,TAATAGAGACAAAGACACCAGCT
5,TAGTAGATACACAAACACCAGCT,TAATAGAGACAGAGACACCAGCT
6,TAGTAGATACACAAACACCAGCT,TACTAAAGACAAAGACACCAGCT
7,TAGTAGATACACAAACACCAGCT,TACTACAAACAAAGACACCAGCT
8,TAGTAGATACACAAACACCAGCT,TACTACACACAAAGACACCAGCT
9,TAGTAGATACACAAACACCAGCT,TACTACAGACAAAGACACCAGCT


In [42]:
df1 = df_crRNA1[['GuideAPosition', 'pgRNA']].copy()
GuideAPositions = []
pgRNAs = []
for _, row in df1.iterrows():
    GuideAPosition = row.GuideAPosition
    
    for val in row.pgRNA:
        GuideAPositions.append(GuideAPosition)
        pgRNAs.append(val)
df2 = pd.DataFrame({"GuideAPosition": GuideAPositions,
    "pgRNA": pgRNAs
})
df2.head(10)

,GuideAPosition,pgRNA
0,15133,TAATACAGACAAAGACACCAGCT
1,15133,TAATACAGACAGAAACACCAGCT
2,15133,TAATACAGACAGAGACACCAGCT
3,15133,TAATAGACACAAAGACACCAGCT
4,15133,TAATAGAGACAAAGACACCAGCT
5,15133,TAATAGAGACAGAGACACCAGCT
6,15133,TACTAAAGACAAAGACACCAGCT
7,15133,TACTACAAACAAAGACACCAGCT
8,15133,TACTACACACAAAGACACCAGCT
9,15133,TACTACAGACAAAGACACCAGCT


In [43]:
df1 = df_crRNA1[['GuideBPosition', 'pgRNA']].copy()
GuideBPositions = []
pgRNAs = []
for _, row in df1.iterrows():
    GuideBPosition = row.GuideBPosition
    
    for val in row.pgRNA:
        GuideBPositions.append(GuideBPosition)
        pgRNAs.append(val)
df2 = pd.DataFrame({"GuideBPosition": GuideBPositions,
    "pgRNA": pgRNAs
})
df2.head(10)

,GuideBPosition,pgRNA
0,9266,TAATACAGACAAAGACACCAGCT
1,9266,TAATACAGACAGAAACACCAGCT
2,9266,TAATACAGACAGAGACACCAGCT
3,9266,TAATAGACACAAAGACACCAGCT
4,9266,TAATAGAGACAAAGACACCAGCT
5,9266,TAATAGAGACAGAGACACCAGCT
6,9266,TACTAAAGACAAAGACACCAGCT
7,9266,TACTACAAACAAAGACACCAGCT
8,9266,TACTACACACAAAGACACCAGCT
9,9266,TACTACAGACAAAGACACCAGCT


In [44]:
df1 = df_crRNA1[['GuideAScore', 'pgRNA']].copy()
GuideAScores = []
pgRNAs = []
for _, row in df1.iterrows():
    GuideAScore = row.GuideAScore
    
    for val in row.pgRNA:
        GuideAScores.append(GuideAScore)
        pgRNAs.append(val)
df2 = pd.DataFrame({"GuideAScore": GuideAScores,
    "pgRNA": pgRNAs
})
df2.head(10)

,GuideAScore,pgRNA
0,1.0,TAATACAGACAAAGACACCAGCT
1,1.0,TAATACAGACAGAAACACCAGCT
2,1.0,TAATACAGACAGAGACACCAGCT
3,1.0,TAATAGACACAAAGACACCAGCT
4,1.0,TAATAGAGACAAAGACACCAGCT
5,1.0,TAATAGAGACAGAGACACCAGCT
6,1.0,TACTAAAGACAAAGACACCAGCT
7,1.0,TACTACAAACAAAGACACCAGCT
8,1.0,TACTACACACAAAGACACCAGCT
9,1.0,TACTACAGACAAAGACACCAGCT


In [45]:
df1 = df_crRNA1[['GuideBScore', 'pgRNA']].copy()
GuideBScores = []
pgRNAs = []
for _, row in df1.iterrows():
    GuideBScore = row.GuideBScore
    
    for val in row.pgRNA:
        GuideBScores.append(GuideBScore)
        pgRNAs.append(val)
df_crRNA_expanded = pd.DataFrame({"GuideA": GuideAs, "GuideB": GuideBs,
    "pgRNA": pgRNAs
})
df_crRNA_expanded.head(10)

,GuideA,GuideB,pgRNA
0,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATACAGACAAAGACACCAGCT
1,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATACAGACAGAAACACCAGCT
2,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATACAGACAGAGACACCAGCT
3,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATAGACACAAAGACACCAGCT
4,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATAGAGACAAAGACACCAGCT
5,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATAGAGACAGAGACACCAGCT
6,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTAAAGACAAAGACACCAGCT
7,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTACAAACAAAGACACCAGCT
8,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTACACACAAAGACACCAGCT
9,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTACAGACAAAGACACCAGCT


In [46]:
df_crRNA_expanded["GuideScore"] = Guide_score_list
df_crRNA_expanded["GuideScore_At_TargetA"] = Guide_scoreA_list
df_crRNA_expanded["GuideScore_At_TargetB"] = Guide_scoreB_list
df_crRNA_expanded["GuideAScore"] = GuideAScores
df_crRNA_expanded["GuideBScore"] = GuideBScores
df_crRNA_expanded["GuideAPosition"] = GuideAPositions
df_crRNA_expanded["GuideBPosition"] = GuideBPositions


In [47]:
# deleting df_crRNA1 dataframe
del df_crRNA1

In [48]:
df_crRNA1 = df_crRNA_expanded

In [49]:
### Adding a column for the GC content
GC_content_list = df_crRNA1["pgRNA"].apply(gc_content)

In [50]:
df_crRNA1["GC_content%"] = GC_content_list

In [51]:
df_crRNA1

,GuideA,GuideB,pgRNA,GuideScore,GuideScore_At_TargetA,GuideScore_At_TargetB,GuideAScore,GuideBScore,GuideAPosition,GuideBPosition,GC_content%
0,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATACAGACAAAGACACCAGCT,0.682070,0.788025,0.576115,1.000000,0.932930,15133,9266,39.130435
1,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATACAGACAGAAACACCAGCT,0.566662,0.558761,0.574564,1.000000,0.932930,15133,9266,39.130435
2,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATACAGACAGAGACACCAGCT,0.571337,0.572266,0.570408,1.000000,0.932930,15133,9266,43.478261
3,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATAGACACAAAGACACCAGCT,0.553227,0.562790,0.543663,1.000000,0.932930,15133,9266,39.130435
4,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATAGAGACAAAGACACCAGCT,0.683648,0.762638,0.604658,1.000000,0.932930,15133,9266,39.130435
5,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATAGAGACAGAGACACCAGCT,0.576249,0.553830,0.598669,1.000000,0.932930,15133,9266,43.478261
6,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTAAAGACAAAGACACCAGCT,0.603838,0.647122,0.560555,1.000000,0.932930,15133,9266,39.130435
7,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTACAAACAAAGACACCAGCT,0.610623,0.638558,0.582688,1.000000,0.932930,15133,9266,39.130435
8,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTACACACAAAGACACCAGCT,0.631835,0.713384,0.550287,1.000000,0.932930,15133,9266,43.478261
9,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTACAGACAAAGACACCAGCT,0.789366,0.966707,0.612025,1.000000,0.932930,15133,9266,43.478261


#### Doing Multiple sequences call in RNAfold

In [52]:
# Initializing list  
list1 = df_crRNA1['pgRNA'].values.tolist()
  
len(list1)

29814

In [53]:
# Criteria 3
# Min Free energy of hainpin + pgRNA > -5
##Adding the hairpin structure to the pgRNAs in the final list2
list3 = []
for val in list1:
    Hainpn = "AACCCCTACCAACTGGTCGGGGTTTGAAAC"
    crRNA_combined = Hainpn + val
    list3.append(crRNA_combined)

In [54]:
# Converting the list3 to a text file
with open('crRNA_combined.txt', 'w') as f:
    for item in list3:
        f.write("%s\n" % item)

In [55]:
# Converting the text file to a fasta file

fileInput = open("crRNA_combined.txt", "r")
fileOutput = open("out_crRNA_combined.fsa", "w")
#Seq count
count = 1 ;

#Loop through each line in the input file
print("Converting to FASTA...")
for strLine in fileInput:

    #Strip the endline character from each input line
    strLine = strLine.rstrip("\n")

    #Output the header
    fileOutput.write(">" + str(count) + "\n")
    fileOutput.write(strLine + "\n")

    count = count + 1
print ("Done.")

#Close the input and output file
fileInput.close()
fileOutput.close()

Converting to FASTA...
Done.


In [56]:
### Function to calculate the Minimum Free energy 
def cal_Min_Eng(x):
    s = x.split("\n")[0]
    s_negative = s.find("-")
    numeric_filter = filter(str.isdigit, s)
    numeric_string = "".join(numeric_filter)
    l = list(numeric_string)
    length_l = len(l)
    l.insert(length_l-2, ".")
    Mng = ''.join(l)
    Mng = float(Mng)
    if (s_negative == -1):
        Mng = Mng
    else:
        Mng = -Mng
        
    return Mng

In [57]:
### Function to calculate the Secondary Structure Using RNAFold Software
def cal_Structure(s):
    s1 = s.split(" ")
    Structure = s1[0]
    return Structure

In [58]:
command1 = "RNAfold < out_crRNA_combined.fsa >> energyoutput.txt"
subprocess.call(command1, shell=True) 

0

In [59]:
command2 = "del *_ss.ps"
subprocess.call(command2, shell=True) 

0

In [60]:
f=open("energyoutput.txt", "r")
contents =f.readlines()
l = len(contents)
sequence_count = []
energy_count = []
for i in range(l):
    For_Query = contents[i].find(">")
    #print(For_Query)
    if (For_Query == 0):
        sequence_count.append(contents[i+1])
        energy_count.append(contents[i+2])
        
f.close()

In [61]:
energy_list = []
Structure_list = []
for val in energy_count:
    energy = cal_Min_Eng(val)
    energy_list.append(energy)
    Structure = cal_Structure(val)
    Structure_list.append(Structure)

In [62]:
df_crRNA1["Free Energy"] = energy_list
df_crRNA1["Secondary Structure"] = Structure_list

In [63]:
HairpinNRG = -12.8
df_crRNA2 = df_crRNA1.loc[(df_crRNA1['Free Energy']- HairpinNRG) > -5]
len( df_crRNA2)

28698

In [64]:
df_crRNA2 = df_crRNA2.reset_index(drop=True)
df_crRNA2

,GuideA,GuideB,pgRNA,GuideScore,GuideScore_At_TargetA,GuideScore_At_TargetB,GuideAScore,GuideBScore,GuideAPosition,GuideBPosition,GC_content%,Free Energy,Secondary Structure
0,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATACAGACAAAGACACCAGCT,0.682070,0.788025,0.576115,1.000000,0.932930,15133,9266,39.130435,-12.8,((((((.(((....))).)))))).........................
1,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATACAGACAGAAACACCAGCT,0.566662,0.558761,0.574564,1.000000,0.932930,15133,9266,39.130435,-12.8,((((((.(((....))).)))))).........................
2,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATACAGACAGAGACACCAGCT,0.571337,0.572266,0.570408,1.000000,0.932930,15133,9266,43.478261,-12.8,((((((.(((....))).)))))).........................
3,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATAGACACAAAGACACCAGCT,0.553227,0.562790,0.543663,1.000000,0.932930,15133,9266,39.130435,-12.8,((((((.(((....))).)))))).........................
4,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATAGAGACAAAGACACCAGCT,0.683648,0.762638,0.604658,1.000000,0.932930,15133,9266,39.130435,-12.8,((((((.(((....))).)))))).........................
5,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TAATAGAGACAGAGACACCAGCT,0.576249,0.553830,0.598669,1.000000,0.932930,15133,9266,43.478261,-12.8,((((((.(((....))).)))))).........................
6,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTAAAGACAAAGACACCAGCT,0.603838,0.647122,0.560555,1.000000,0.932930,15133,9266,39.130435,-12.8,((((((.(((....))).)))))).........................
7,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTACAAACAAAGACACCAGCT,0.610623,0.638558,0.582688,1.000000,0.932930,15133,9266,39.130435,-12.8,((((((.(((....))).)))))).........................
8,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTACACACAAAGACACCAGCT,0.631835,0.713384,0.550287,1.000000,0.932930,15133,9266,43.478261,-12.8,((((((.(((....))).)))))).........................
9,TACTACAGATAGAGACACCAGCT,TAGTAGATACACAAACACCAGCT,TACTACAGACAAAGACACCAGCT,0.789366,0.966707,0.612025,1.000000,0.932930,15133,9266,43.478261,-12.8,((((((.(((....))).)))))).........................


In [65]:
df_crRNA2.to_csv (r'df_crRNA_withEnergy_filtered.csv', index = False, header=True)

In [66]:
list_filtered = df_crRNA2['pgRNA'].values.tolist()
  
len(list_filtered)

28698

In [67]:
reverse_complement_rna_list = []
for val in list_filtered:
    # Taking the reverse complement of pg-DNA sequence
    s = revcom(val)
    s = s.replace('T', 'U')
    reverse_complement_rna_list.append(s)
    
    

In [68]:
len(reverse_complement_rna_list)

28698

In [69]:
# Converting the reverse_complement_rna_list into a text file
with open('pgRNA_filtered.txt', 'w') as f:
    for item in reverse_complement_rna_list:
        f.write("%s\n" % item)

In [70]:
# Converting the text file to a fasta file

fileInput = open("pgRNA_filtered.txt", "r")
fileOutput = open("out_pgRNA_filtered.fsa", "w")
#Seq count
count = 1 ;

#Loop through each line in the input file
print("Converting to FASTA...")
for strLine in fileInput:

    #Strip the endline character from each input line
    strLine = strLine.rstrip("\n")

    #Output the header
    fileOutput.write(">" + str(count) + "\n")
    fileOutput.write(strLine + "\n")

    count = count + 1
print ("Done.")

#Close the input and output file
fileInput.close()
fileOutput.close()

Converting to FASTA...
Done.


### Blast search

In [71]:
command = 'blastn -query out_pgRNA_filtered.fsa -task blastn-short -db C:/Users/18064/Documents/Blast/db/Human_NCBI_rnadb -outfmt "7 qacc sacc qstart qend sstart send" -out pgRNA_NCBI_blast.out'
subprocess.call(command, shell=True)

0

In [72]:
f=open('pgRNA_NCBI_blast.out', "r")
contents =f.readlines()
query_list =[]
hits_list = []
for x in contents:
    For_Query = x.find("Query")
    if (For_Query != -1):
        query_list.append(x)
        #print("For Sequence",x)
    hits_count = x.find("hits found")
    if (hits_count != -1):
        hits_list.append(x)
        #print(x)

# Making a list of query sequence
new_query_list =[]
for val in query_list:
    x = val.split("\n")[0].split("#")[1].split(":")[1].split(" ")[1]
    x = int(x)
    new_query_list.append(x)

# Making a list of blast hits in human genome for the query sequences
new_hit_list =[]
for val in hits_list:
    x = val.split("\n")[0].split(" ")[1]
    x = int(x)
    new_hit_list.append(x)


In [73]:
blast_hits = pd.DataFrame({'GuideNumber': new_query_list,'Hits': new_hit_list})
blast_hits.head()

,GuideNumber,Hits
0,1,61
1,2,41
2,3,13
3,4,8
4,5,30


In [74]:
len(blast_hits)

28698

In [75]:
df_crRNA2["HumanBlastHits"] = new_hit_list

In [76]:
df_crRNA2.to_csv (r'df_crRNA_withhits.csv', index = False, header=True)

In [77]:
f=open("pgRNA_NCBI_blast.out", "r")
contents =f.readlines()
Blast_list = []
for i in range(len(contents)):
    target = contents[i].find("#")
    if (target == -1):
        Blast_list.append(i)

In [78]:
seq_list = []
accession_list = []
qstart_list = []
qend_list = []
sstart_list = []
send_list = []
for val in Blast_list:
    line = contents[val]
    seq = line.split('\t')[0]
    accession = line.split('\t')[1]
    qstart = line.split('\t')[2]
    qend = line.split('\t')[3]
    sstart = line.split('\t')[4]
    send = line.split('\t')[5].split('\n')[0]
    seq_list.append(seq)
    accession_list.append(accession)
    qstart_list.append(qstart)
    qend_list.append(qend)
    sstart_list.append(sstart)
    send_list.append(send)
    
d = {'Sequence':seq_list,'Accession':accession_list,'qstart':qstart_list,'qend':qend_list,'sstart':sstart_list,'send':send_list}
df_blast = pd.DataFrame(d)
df_blast.head()

,Sequence,Accession,qstart,qend,sstart,send
0,1,XM_017009351,2,17,2859,2844
1,1,NM_001017995,2,17,2777,2762
2,1,XR_001740339,4,18,6549,6535
3,1,XR_001740338,4,18,6552,6538
4,1,XM_024454293,9,23,1991,1977


In [79]:
# This is for Cas13d
range_list = []
seq_4_range_list = []
l = len(sstart_list)
for i in range(l):
    if (sstart_list[i] < send_list[i]): # For RNA, hit same strand
        #print('strand = plus')

        range_blast = accession_list[i] + '  '+ str(int(sstart_list[i])-int(qstart_list[i])+1) + '-' + str(int(send_list[i])+ 23 - int(qend_list[i])) + '  '+ 'plus'
        range_list.append(range_blast)
        seq_4_range_list.append(seq_list[i])

In [80]:
range_list

['XR_001739130  1548-1570  plus',
 'XR_001739129  1548-1570  plus',
 'XR_001739127  1548-1570  plus',
 'XR_001739126  1548-1570  plus',
 'XM_024451269  8922-8944  plus',
 'XM_017026014  12427-12449  plus',
 'XM_017026013  12580-12602  plus',
 'XM_017026012  12218-12240  plus',
 'XM_011526213  8983-9005  plus',
 'XM_011526212  8961-8983  plus',
 'XM_011526209  9000-9022  plus',
 'XM_011526206  12371-12393  plus',
 'XM_011526205  15606-15628  plus',
 'XM_005258356  8816-8838  plus',
 'NM_030632  9201-9223  plus',
 'NM_004788  5175-5197  plus',
 'NM_001204077  5154-5176  plus',
 'XM_017002970  2294-2316  plus',
 'XM_017002969  2370-2392  plus',
 'XM_017002968  2007-2029  plus',
 'XM_011510210  2009-2031  plus',
 'XM_011510209  3002-3024  plus',
 'XM_011510208  3140-3162  plus',
 'XM_011510207  2864-2886  plus',
 'XM_011510206  3002-3024  plus',
 'XM_011510205  3140-3162  plus',
 'XM_005245653  2248-2270  plus',
 'XM_005245649  2825-2847  plus',
 'NM_201569  2042-2064  plus',
 'NM_201568  

In [81]:
len(range_list)

220395

In [82]:
#seq_4_range_list

In [83]:
# Converting the list to a text file
with open('range_NCBIBlasthits.txt', 'w') as f:
    for item in range_list:
        f.write("%s\n" % item)

In [84]:
command1 = 'blastdbcmd -db C:/Users/18064/Documents/Blast/db/Human_NCBI_rnadb -entry_batch range_NCBIBlasthits.txt -outfmt "%s" -out NCBI_Human_RNA_Range.txt'
subprocess.call(command1, shell=True)

0

In [85]:
## Reading the NCBI_Human_RNA_Range.txt file
f_rangeRNA=open("NCBI_Human_RNA_Range.txt", "r")
contents_rangeRNA =f_rangeRNA.readlines()
Human_rangeRNA_list = []
for val in contents_rangeRNA:
    val1 = (val.split('\n')[0])[0:23]
    Human_rangeRNA_list.append(val1)   

In [86]:
len(Human_rangeRNA_list)

220395

In [87]:
seq_blast_cfd = []
GuideScore_blast = []
for val in seq_4_range_list:
    i = (int(val) - 1)
    #print(i)
    seq_blast = revcom(df_crRNA2.iloc[i].loc['pgRNA'])
    guide_score = df_crRNA2.iloc[i].loc['GuideScore']
    seq_blast_cfd.append(seq_blast)
    GuideScore_blast.append(guide_score)

In [88]:
Blast_cfd_df = pd.DataFrame({'GuideNumber': seq_4_range_list,'SequenceComplementGuide': seq_blast_cfd,'HumanRNA_range': Human_rangeRNA_list,'GuideActivity': GuideScore_blast})

In [89]:
Blast_cfd_df.head()
    

,GuideNumber,SequenceComplementGuide,HumanRNA_range,GuideActivity
0,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207
1,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207
2,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207
3,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207
4,1,AGCTGGTGTCTTTGTCTGTATTA,CAGTGGTGTCTTTGTCTGCCTGG,0.68207


In [91]:
## Calculating the cfd score for the Human transcriptome region corresponding to blast hits
x = len(Human_rangeRNA_list)
cfd_humanBlast_list = []
Activity_humanBlast_list = []
for i in range(x):
    #print(i)
    try:
        cfd_humanBlast = Cal_Mismatch_Score(Blast_cfd_df["SequenceComplementGuide"][i], Blast_cfd_df["HumanRNA_range"][i])
        Activity_humansite = cfd_humanBlast*Blast_cfd_df["GuideActivity"][i]
    except IndexError:
        cfd_humanBlast = 'null'
        Activity_humansite = "null"
    cfd_humanBlast_list.append(cfd_humanBlast)
    Activity_humanBlast_list.append(Activity_humansite)

In [92]:
Blast_cfd_df["CFD_Score"] = cfd_humanBlast_list
Blast_cfd_df["ActivityScore"] = Activity_humanBlast_list

In [93]:
Blast_cfd_df.head()

,GuideNumber,SequenceComplementGuide,HumanRNA_range,GuideActivity,CFD_Score,ActivityScore
0,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207,0.046602,0.0317858
1,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207,0.046602,0.0317858
2,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207,0.046602,0.0317858
3,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207,0.046602,0.0317858
4,1,AGCTGGTGTCTTTGTCTGTATTA,CAGTGGTGTCTTTGTCTGCCTGG,0.68207,0.508878,0.34709


In [94]:
Blast_cfd_df["Guide"] = Blast_cfd_df["SequenceComplementGuide"].apply(revcom)

In [95]:
Blast_cfd_df.head()

,GuideNumber,SequenceComplementGuide,HumanRNA_range,GuideActivity,CFD_Score,ActivityScore,Guide
0,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207,0.046602,0.0317858,TAATACAGACAAAGACACCAGCT
1,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207,0.046602,0.0317858,TAATACAGACAAAGACACCAGCT
2,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207,0.046602,0.0317858,TAATACAGACAAAGACACCAGCT
3,1,AGCTGGTGTCTTTGTCTGTATTA,AGCTGGTGTCTTTGTTCTCCAGT,0.68207,0.046602,0.0317858,TAATACAGACAAAGACACCAGCT
4,1,AGCTGGTGTCTTTGTCTGTATTA,CAGTGGTGTCTTTGTCTGCCTGG,0.68207,0.508878,0.34709,TAATACAGACAAAGACACCAGCT


In [96]:
Blast_cfd_df.to_csv (r'BlastHumanRnaHits_Activity.csv', index = False, header=True)

In [97]:
end = time.time()
print(end - start)

9625.920943498611
